In [3]:
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from pandas import DataFrame

In [15]:
"""
S = Current Stock Price
K = Strike price
t = time to maturity
r = risk-free rate
sigma = volatility
"""
def d1(S,K,t,r,sigma):
    return (1/sigma*sqrt(t))*(log(S/K)+(r+(sigma**2)/2)*t)
def d2(S,K,t,r,sigma):
    return d1(S,K,t,r,sigma)-sigma*sqrt(t)

In [16]:
# european call
def bs_call(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))

# european put
def bs_put(S,K,T,r,sigma):
    return K*exp(-r*T)-S+bs_call(S,K,T,r,sigma)

In [25]:
#stock = '^GDAXI'
expiry = '2022-12-14'
strike_price = 160
relation = 0.1 #Bezugsverhältnis

today = datetime(2022, 5, 19)
#print(today)
one_year_ago = today.replace(year=today.year-1)

In [18]:
dax_40 = {
    "adidas" : web.DataReader("ADS.DE", 'yahoo', one_year_ago, today),
    "airbus" : web.DataReader("AIR.DE", 'yahoo', one_year_ago, today),
    "allianz" : web.DataReader("ALV.DE", 'yahoo', one_year_ago, today),
    "basf" : web.DataReader("BAS.DE", 'yahoo', one_year_ago, today),
}

for k in dax_40.keys():
    dax_40[k] = dax_40[k].sort_values(by="Date")
    dax_40[k] = dax_40[k].dropna()
    dax_40[k] = dax_40[k].assign(close_day_before=dax_40[k].Close.shift(1))
    dax_40[k]['returns'] = ((dax_40[k].Close - dax_40[k].close_day_before)/dax_40[k].close_day_before)
dax_40["adidas"]

,High,Low,Open,Close,Volume,Adj Close,close_day_before,returns
Date,,,,,,,,
2021-05-19,295.350006,289.350006,293.049988,294.549988,629842.0,289.164276,NaN,NaN
2021-05-20,298.399994,293.899994,297.000000,298.399994,499980.0,292.943878,294.549988,0.013071
2021-05-21,301.950012,299.100006,300.000000,301.950012,677856.0,296.428986,298.399994,0.011897
2021-05-25,304.399994,299.149994,304.399994,299.149994,555295.0,293.680176,301.950012,-0.009273
2021-05-26,299.350006,294.450012,299.000000,297.049988,515046.0,291.618561,299.149994,-0.007020
...,...,...,...,...,...,...,...,...
2022-05-13,180.479996,180.479996,180.479996,180.479996,0.0,180.479996,180.479996,0.000000
2022-05-16,181.679993,177.820007,179.460007,180.059998,648134.0,180.059998,180.479996,-0.002327
2022-05-17,186.820007,180.440002,181.899994,183.300003,630018.0,183.300003,180.059998,0.017994


In [26]:
company = "adidas"

df = dax_40[company]

sigma = np.sqrt(252) * df['returns'].std()
uty = (web.DataReader(
    "^TNX", 'yahoo', today.replace(day=today.day-1), today)['Close'].iloc[-1])/100
lcp = df['Close'].iloc[-1]
t = (datetime.strptime(expiry, "%Y-%m-%d") - datetime.utcnow()).days / 365


print("+++ " + str(company) + " +++")
print('The Put Option Price is: ', bs_put(lcp, strike_price, t, uty, sigma)*relation)
print('The Call Option Price is: ', bs_call(lcp, strike_price, t, uty, sigma)*relation)

+++ adidas +++
The Put Option Price is:  1.0166852779302702
The Call Option Price is:  2.4608296454969776


In [14]:
t = (datetime.strptime(expiry, "%Y-%m-%d") - datetime.utcnow()).days / 365
print(t)

0.5698630136986301
